In [2]:
%matplotlib inline

import seaborn as sns
import pandas as pd
import datetime as dt
import random
import itertools
import pdb
import ipdb
import time
from dateutil.parser import parse
from experiment.models import Models, Parameters
from experiment.models import get_metadata_file, get_pubdate_index

mt = get_metadata_file()
p = Parameters()
p.q = "Mitch Landrieu"

start_time = time.time()
results = Models.get_results(p)
print "getting results took {}".format(start_time - time.time())

start_time = time.time()
facets, aliases = Models.get_facets(p, results, 3)
print "facets took {}".format(start_time - time.time())

metadata = [mt[r] for r in results]
q_pubdates = [parse(h["pubdate"]) for h in metadata]

def make_dataframe(p, facets, results):
    start_time = time.time()
    df = pd.DataFrame()
    df['id'] = results
    df['pd'] = q_pubdates
    df[p.q] = [1 for i in q_pubdates]
    for facet in facets:
        alias = [pub_index[a] for a in aliases[facet]]
        facet_dates = set([i for i in set([i for i in itertools.chain(*alias)]).union(pub_index[facet])])
        gus = [int(p) for p in [p in facet_dates for p in q_pubdates]]
        df[facet] = gus
    print "dataframe took {}".format(start_time - time.time())
    return df

df = make_dataframe(p, facets, results)
df = df.groupby([df['pd'].map(lambda x: x.year)]).sum().unstack(0).fillna(0)

facet_datas = []
for f in facets:
    print facet_datas.append([str(f)] + list(df[f]))
print df[p.q]
    
print facet_datas

DEBUG:experiment:got facets. time to filter


getting results took -1.55303788185
facets took -1.67511987686
dataframe took -0.00319695472717
None
None
None
pd
2010    84
2011    58
2012    26
2013    62
2014    65
2015    17
dtype: int64
[['Department of Justice', 12, 6, 7, 20, 4, 0], ['Marlin Gusman', 31, 23, 8, 20, 15, 1], ['Orleans Parish Sheriffs Office', 5, 14, 4, 17, 8, 2]]


In [1]:
from experiment.models import get_metadata_file, get_pubdate_index
pub_index = get_pubdate_index()

loading pubdate index
built index
